In [ ]:
pip install datasets torch numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
!pip install transformers datasets torchaudio soundfile
!pip install git+https://github.com/huggingface/transformers.git  # 确保最新版

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-8uo_ax00
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-8uo_ax00
  Resolved https://github.com/huggingface/transformers.git to commit 51d732709e5ae424e8fb6c4e58b72057a3e413c2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.53.0.dev0-py3-none-any.whl size=11363407 sha256=84db875625ed4ee6fc2684f44b7305ec436822a45988202f755d292281f3ecda
  Stored in directory: /tmp/pip-ephem-wheel-cache-c2g6flaw/wheels/32/4b/78/f195c684dd3a9ed21f3b39fe8f85b48df7918581b6437be143
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.2
    Uninstalling transformers-4.52.2:
      Successfully uninstalled transformers-4.52.2


In [11]:
import torch
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
import soundfile as sf
from IPython.display import Audio
import numpy as np
import os
import zipfile
import requests

# 1. 清理缓存
!rm -rf ~/.cache/huggingface/datasets

# 2. 下载并解压数据集（确保完整下载）
dataset_url = "https://huggingface.co/datasets/Matthijs/cmu-arctic-xvectors/resolve/main/spkrec-xvect.zip"
download_path = "spkrec-xvect.zip"
extract_path = "embeddings"

# 创建目录
os.makedirs(extract_path, exist_ok=True)

# 下载文件（显示进度）
if not os.path.exists(download_path):
    print("正在下载数据集...")
    response = requests.get(dataset_url, stream=True)
    total_size = int(response.headers.get('content-length', 0))

    with open(download_path, 'wb') as f:
        downloaded = 0
        for data in response.iter_content(chunk_size=1024):
            downloaded += len(data)
            f.write(data)
            # 显示进度
            print(f"下载进度: {downloaded/total_size*100:.1f}%", end='\r')
    print("\n下载完成!")

# 解压文件
if not os.listdir(extract_path):
    print("正在解压文件...")
    with zipfile.ZipFile(download_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("解压完成!")

# 3. 获取嵌入文件列表
embedding_files = []
for root, dirs, files in os.walk(extract_path):
    for file in files:
        if file.endswith(".npy"):
            embedding_files.append(os.path.join(root, file))

# 检查文件数量
if len(embedding_files) == 0:
    raise RuntimeError("未找到任何嵌入文件，请检查下载和解压过程")

print(f"找到 {len(embedding_files)} 个说话人嵌入文件")

# 4. 加载模型
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")



找到 7931 个说话人嵌入文件


In [29]:
# 尝试第100个说话人（确保索引小于文件总数）
new_index = min(200, len(embedding_files)-1)  # 安全索引
sample_file = embedding_files[new_index]

# 重新加载嵌入
speaker_embedding = torch.tensor(np.load(sample_file)).unsqueeze(0)

In [30]:

# 6. 生成语音
text = "Memento mori!!!"

# 文本预处理
inputs = processor(text=text, return_tensors="pt")

# 生成语音（使用GPU加速）
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
vocoder = vocoder.to(device)


with torch.no_grad():
    speech = model.generate_speech(
        inputs["input_ids"].to(device),
        speaker_embeddings=speaker_embedding,
        vocoder=vocoder
    )

# 7. 保存并播放结果
speech_np = speech.cpu().numpy().squeeze()
sf.write("output.wav", speech_np, samplerate=16000)
print("\n生成完成! 播放音频:")
Audio("output.wav", autoplay=True)



生成完成! 播放音频:


In [31]:
import os

# 设置从https://hf-mirror.com下载模型，否则会从huggingface.co下载
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

import torch
import librosa # pip install librosa
from transformers import pipeline

# 加载音频文件
audio, sample_rate = librosa.load("/content/output.wav")

# 处理音频数据
pipeline = pipeline(
    task="automatic-speech-recognition",
    model="openai/whisper-small",
)

# 输出结果
result = pipeline(audio)
print(result)

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

Device set to use cpu
/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:582: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.


{'text': ' Memento Mori.'}
